In [66]:
import json
import pandas as pd
import numpy as np

In [1]:
from predectorutils.analyses import EffectorDB

In [19]:
edb = pd.read_csv("input/effectordb.tsv", sep="\t")

In [16]:
matches = []
with open("output/predector_protein/effectordb_matches.ldjson", "r") as handle:
    for line in handle:
        j = json.loads(line)
        match = EffectorDB.from_dict(j["data"])
        if match.decide_significant():
            matches.append(match.as_series())

matches = pd.DataFrame(matches)

In [17]:
matches

,query,hmm,hmm_len,query_len,full_evalue,full_score,full_bias,nmatches,domain_c_evalue,domain_i_evalue,domain_score,domain_bias,hmm_from,hmm_to,query_from,query_to,acc,description
0,sscle_01g000430,ED_000523,270,1058,1.400000e-10,36.0,17.5,5,1.600000e-12,1.400000e-10,36.0,17.5,94,215,10,130,0.75,None
1,sscle_01g000430,ED_000077,132,1058,4.500000e-09,31.3,21.1,3,7.100000e-11,6.000000e-09,30.9,2.8,18,85,20,93,0.77,None
2,sscle_01g000430,ED_000500,110,1058,6.200000e-05,18.4,3.3,8,7.400000e-07,6.200000e-05,18.4,3.3,44,100,8,68,0.80,None
3,sscle_01g000430,ED_000151,120,1058,6.400000e-05,18.0,2.7,3,7.600000e-07,6.400000e-05,18.0,2.7,35,106,24,99,0.75,None
4,sscle_01g000430,ED_000124,117,1058,3.900000e-04,15.2,0.9,2,4.700000e-06,3.900000e-04,15.2,0.9,51,96,37,87,0.81,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763,sscle_16g111150,ED_000093,185,244,1.200000e-26,88.1,94.0,2,3.600000e-26,2.600000e-24,80.4,52.3,0,164,0,165,0.91,None
1764,sscle_16g111150,ED_000102,173,244,9.800000e-21,68.8,96.0,2,8.200000e-18,5.900000e-16,53.3,63.5,0,158,0,163,0.65,None
1765,sscle_16g111150,ED_000656,184,244,2.100000e-20,68.2,33.7,2,2.900000e-22,2.100000e-20,68.2,33.7,0,134,0,133,0.88,None
1766,sscle_16g111150,ED_000008,147,244,9.000000e-19,62.6,29.2,4,1.300000e-20,9.000000e-19,62.6,29.2,46,142,14,108,0.91,None


In [20]:
edb

,id,organism,taxid,gene,names,notes,reference_urls,uniprot,Genbank,PHI-base,PDB,sequence
0,ED_000001,Bipolaris maydis,5016,ChToxA,ToxA-like protein,"ToxA-like, (formerly N4WQY3, XP_014072442.1)",http://dx.doi.org/10.1016/j.fgb.2015.05.013,A0A0G2RQA9,AJI77148.1,PHI:4924,NaN,MWAPSILLILILGLVQAKPLSLNARELSLDGRELSLDARSVLHKRQ...
1,ED_000002,Blumeria graminis,34373,BgAVRa1,BgAVRa1;CSEP0008;BghAvr_a1,Recognised by Mla1. Triggers HR.,http://dx.doi.org/10.1073/pnas.1612947113,N1JGD1,CCU81904,NaN,NaN,MKFYHSSFVSNLTILSLFLNINGAFSRTWQCPSGDILYEEDVYSHT...
2,ED_000003,Blumeria graminis,34373,BgAVRA13,BgAVRA13;CSEP0372;BghAvr_a13,Recognised by Mla13. Triggers HR.,http://dx.doi.org/10.1073/pnas.1612947113,N1JFM8,CCU76657,NaN,NaN,MKTFQFASIVAGLSFLKTTIAAGDGYITLGMGSIHKNDIYRVAEHM...
3,ED_000004,Blumeria graminis f. sp. hordei,62688,BghBEC1019,BEC1019,"Appears to be broadly conserved across taxon, ...",http://dx.doi.org/10.1094/MPMI-02-15-0027-FI;h...,A0A059UDR8,AHZ59730,PHI:9472;PHI:2897,NaN,MQSVLLLTVLTQSFIATASPLVERSTPLSFAEKRPQKVSYDWTTPY...
4,ED_000008,Blumeria graminis f. sp. hordei,62688,BghBEC2,BEC2;CSEP0214,Contains CFEM domain so likely to be extracell...,http://dx.doi.org/10.1111/mpp.12110,G9BER8,AEQ16460,NaN,NaN,MQLIRLLVLAFAAFVAASPYSTLSHTEGGKGGKGGKGDDDGDDDGP...
...,...,...,...,...,...,...,...,...,...,...,...,...
501,ED_000709,Sclerotinia sclerotiorum,665079,SsINE5,sscle_05g045060,Found intracellularly. Elicits host cell death.,https://doi.org/10.1111/mpp.13333,A0A1D9Q4B1,XP_001593146,NaN,NaN,MVRIASIALFFIASALSVQACTYCQCEFSNGGHCCVYSDAEIGNLD...
502,ED_000710,Sclerotinia sclerotiorum,665079,SsINE1,sscle_02g012940,Found intracellularly. Elicits host cell death.,https://doi.org/10.1111/mpp.13333,A0A1D9PUX6,XP_001586202,NaN,NaN,MKLSAMFILVASLNVIAVPLAHDDSIQRRANFKRTLTDASVDFDVT...
503,ED_000711,Sclerotinia sclerotiorum,665079,SsERP1,SS1G_11468 ; sscle_07g060180,ethylene pathway repressor protein 1. knockout...,https://doi.org/10.3390/jof7100825 ; https://d...,A0A1D9Q8H2; A7F1J8,XP_001587476,NaN,NaN,MRYTIVIAAIIPFVAAHGKIAVLSGDMGGNTTGLGIQGAVIPGAGT...
504,ED_000712,Sclerotinia sclerotiorum,665079,SsPINE1,SS1G_08128 ; sscle_10g074920,PGIP-inactivating effector 1. Deletion mutants...,https://doi.org/10.1038/s41467-022-29788-2,A7ES23,XP_001590388,NaN,NaN,MKLSQPLSIFAILAASTVAVAVPQDDACAQCLTEGTFCEGFAGPVG...


In [22]:
matches = pd.merge(edb, matches.rename(columns={"hmm": "id"}), on="id")

In [23]:
matches.head()

,id,organism,taxid,gene,names,notes,reference_urls,uniprot,Genbank,PHI-base,PDB,sequence,query,hmm_len,query_len,full_evalue,full_score,full_bias,nmatches,domain_c_evalue,domain_i_evalue,domain_score,domain_bias,hmm_from,hmm_to,query_from,query_to,acc,description
0,ED_000003,Blumeria graminis,34373,BgAVRA13,BgAVRA13;CSEP0372;BghAvr_a13,Recognised by Mla13. Triggers HR.,http://dx.doi.org/10.1073/pnas.1612947113,N1JFM8,CCU76657,NaN,NaN,MKTFQFASIVAGLSFLKTTIAAGDGYITLGMGSIHKNDIYRVAEHM...,sscle_13g092480,122,154,7.500000e-06,21.1,0.0,1,1.300000e-07,9.100000e-06,20.8,0.0,26,89,10,81,0.79,None
1,ED_000004,Blumeria graminis f. sp. hordei,62688,BghBEC1019,BEC1019,"Appears to be broadly conserved across taxon, ...",http://dx.doi.org/10.1094/MPMI-02-15-0027-FI;h...,A0A059UDR8,AHZ59730,PHI:9472;PHI:2897,NaN,MQSVLLLTVLTQSFIATASPLVERSTPLSFAEKRPQKVSYDWTTPY...,sscle_10g077990,316,314,8.700000e-108,355.7,4.2,1,5.800000e-110,9.700000e-108,355.5,4.2,0,316,0,314,0.94,None
2,ED_000004,Blumeria graminis f. sp. hordei,62688,BghBEC1019,BEC1019,"Appears to be broadly conserved across taxon, ...",http://dx.doi.org/10.1094/MPMI-02-15-0027-FI;h...,A0A059UDR8,AHZ59730,PHI:9472;PHI:2897,NaN,MQSVLLLTVLTQSFIATASPLVERSTPLSFAEKRPQKVSYDWTTPY...,sscle_10g079810,316,354,3.300000e-53,176.4,15.0,1,3.100000e-55,5.200000e-53,175.7,15.0,21,239,148,353,0.91,None
3,ED_000004,Blumeria graminis f. sp. hordei,62688,BghBEC1019,BEC1019,"Appears to be broadly conserved across taxon, ...",http://dx.doi.org/10.1094/MPMI-02-15-0027-FI;h...,A0A059UDR8,AHZ59730,PHI:9472;PHI:2897,NaN,MQSVLLLTVLTQSFIATASPLVERSTPLSFAEKRPQKVSYDWTTPY...,sscle_13g096800,316,685,2.500000e-20,68.4,4.9,3,4.800000e-22,6.100000e-20,67.1,0.1,91,262,155,345,0.73,None
4,ED_000004,Blumeria graminis f. sp. hordei,62688,BghBEC1019,BEC1019,"Appears to be broadly conserved across taxon, ...",http://dx.doi.org/10.1094/MPMI-02-15-0027-FI;h...,A0A059UDR8,AHZ59730,PHI:9472;PHI:2897,NaN,MQSVLLLTVLTQSFIATASPLVERSTPLSFAEKRPQKVSYDWTTPY...,sscle_14g100080,316,354,1.200000e-41,138.4,8.9,1,1.200000e-43,2.000000e-41,137.7,8.9,22,237,154,348,0.86,None


In [39]:
effector_matches = (
    matches
    .drop(columns=["sequence", "full_bias", "acc", "description", "nmatches", "domain_bias"])
    .rename(columns={
        "Genbank": "genbank",
        "PHI-base": "phibase",
        "PDB": "pdb",
        "query": "gene_name",
        "id": "edb_id",
        "hmm_from": "edb_start",
        "hmm_to": "edb_end",
        "hmm_len": "edb_length",
        "query_from": "gene_start",
        "query_to": "gene_end",
        "query_len": "gene_length",
        "gene": "effector_name",
        "names": "effector_altnames",
    })
    [[
        'gene_name',
        'edb_id',
        'effector_name',
        'effector_altnames',
        'organism',
        'taxid',
        'notes',
        'reference_urls',
        'uniprot',
        'genbank',
        'phibase',
        'pdb',
        'edb_start',
        'edb_end',
        'edb_length',
        'gene_start',
        'gene_end',
        'gene_length',
        'full_evalue',
        'full_score',
        'domain_c_evalue',
        'domain_i_evalue',
        'domain_score',
    ]]
    .sort_values(["gene_name", "full_score"], ascending = False)
)

effector_matches.to_csv("output/predector_protein/effectordb_matches.tsv", sep="\t", na_rep="-", index=False)
effector_matches.head()

,gene_name,edb_id,effector_name,effector_altnames,organism,taxid,notes,reference_urls,uniprot,genbank,phibase,pdb,edb_start,edb_end,edb_length,gene_start,gene_end,gene_length,full_evalue,full_score,domain_c_evalue,domain_i_evalue,domain_score
533,sscle_16g111290,ED_000118,MlAvrL2,AvrL2,Melampsora lini,5261,AvrL2-A is virulent form. Three other forms de...,http://dx.doi.org/10.1186/s12864-016-3011-9,A0A1B2CW13,ANY58853,NaN,NaN,64,141,150,12,89,150,4.100000e-20,67.3,1.100000e-22,5.600000e-20,66.8
50,sscle_16g111150,ED_000010,BghBEC4,BEC4,Blumeria graminis f. sp. hordei,62688,Interacts with ADP ribosylation factor-GTPase-...,http://dx.doi.org/10.1111/mpp.12110;http://dx....,G9BES0,AEQ16462.1,NaN,NaN,3,233,235,0,243,244,2.900000e-30,99.9,6.400000e-32,4.600000e-30,99.3
409,sscle_16g111150,ED_000093,MoCDIP11,MoCDIP11; MGG_12521,Magnaporthe oryzae,318829,"CDIP, CFEM domain",http://dx.doi.org/10.1186/s12284-019-0312-z,G5EHP3,XP_003720604.1;XM_003720556.1,NaN,NaN,0,164,185,0,165,244,1.200000e-26,88.1,3.600000e-26,2.600000e-24,80.4
471,sscle_16g111150,ED_000102,MoCDIP2,MoCDIP2; MGG_05531,Magnaporthe oryzae 7015,242507,Probably apoplastic,http://dx.doi.org/10.1094/MPMI-05-12-0117-R,G4MML4,EHA57786,PHI:3214,NaN,0,158,173,0,163,244,9.800000e-21,68.8,8.200000e-18,5.900000e-16,53.3
1560,sscle_16g111150,ED_000656,Cccopsin,Cccopsin,Coprinopsis cinerea,5346,NaN,https://dx.doi.org/10.1074/jbc.m114.599878;htt...,A0A097PTA8,AIU55999.1,NaN,NaN,0,134,184,0,133,244,2.100000e-20,68.2,2.900000e-22,2.100000e-20,68.2


In [58]:
matches_onecol = (
    effector_matches
    [~pd.isna(effector_matches["effector_name"])]
    .sort_values("full_score", ascending = False)
    .groupby("gene_name effector_name".split())
    .head(1)
    .sort_values("gene_name full_score".split(), ascending = False)
    .groupby("gene_name")
    ["effector_name"].unique().apply(lambda s: "|".join(s)).to_dict()
)

In [68]:
preds = pd.read_csv("./effector_preds.tsv", sep="\t", na_values=".")
preds["effectordb_name"] = preds["name"].apply(lambda x: matches_onecol.get(x, np.nan))
preds

,name,effector_score,phibase_genes,phibase_phenotypes,dbcan_accession,dbcan_description,swissprot_protein_name,swissprot_gene_name,swissprot_accession,pdb_protein_name,pdb_accession,go_accession,go_name,ipr_accession,ipr_name,effectorp1,effectorp2,effectorp3_cytoplasmic,effectorp3_apoplastic,effectorp3_noneffector,deepredeff_fungi,deepredeff_oomycete,apoplastp,molecular_weight,residue_number,charge,isoelectric_point,aa_c_number,aa_tiny_number,aa_small_number,aa_aliphatic_number,aa_aromatic_number,aa_nonpolar_number,aa_charged_number,aa_basic_number,aa_acidic_number,fykin_gap,kex2_cutsites,rxlr_like_motifs,localizer_nucleus,...,phobius_tm_domains,tmhmm_tmcount,tmhmm_first_60,tmhmm_exp_aa,tmhmm_first_tm_sp_coverage,tmhmm_domains,targetp_secreted_prob,targetp_mitochondrial_prob,deeploc_membrane,deeploc_nucleus,deeploc2_nucleus,deeploc_cytoplasm,deeploc2_cytoplasm,deeploc_mitochondrion,deeploc2_mitochondrion,deeploc_plastid,deeploc2_plastid,deeploc_endoplasmic_reticulum,deeploc2_endoplasmic_reticulum,deeploc_golgi,deeploc2_golgi,deeploc_lysosome,deeploc2_lysosome,deeploc_peroxisome,deeploc2_peroxisome,deeploc_cell_membrane,deeploc2_cell_membrane,deeploc_extracellular,deeploc2_extracellular,deeploc2_preds,deeploc2_signals,signalp3_nn_d,signalp3_hmm_s,signalp4_d,signalp5_prob,signalp6_prob,deepsig_signal_prob,deepsig_transmembrane_prob,deepsig_other_prob,effectordb_name
0,sscle_03g031910,3.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,0.97,0.70,0.81,NaN,0.61,0.67,0.42,24242.45,222,2.5,7.633,5,72,124,66,22,121,50,27,23,1.185,LQKR:[LIJVAP]X[KRTPEI]R&LXXR:66-69|KR:[KR]R:68...,KRIA:81-84|KNLK:114-117|KGYH:152-155,0,...,NaN,0,8.77,16.65,0.0,NaN,1.0,0.00,0.00,0.00,0.07,0.0,0.07,0.00,0.06,0.0,0.01,0.00,0.08,0.00,0.08,0.01,0.07,0.0,0.00,0.00,0.10,0.98,0.93,Extracellular,Signal peptide,0.776,0.996,0.677,0.990,1.000,1.0,NaN,NaN,NaN
1,sscle_16g107730,3.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.96,0.71,0.80,NaN,0.61,0.67,0.42,24212.42,222,2.5,7.633,5,72,124,67,22,122,50,27,23,1.164,LQKR:[LIJVAP]X[KRTPEI]R&LXXR:66-69|KR:[KR]R:68...,KRIA:81-84|KNLK:114-117|KGYH:152-155,0,...,NaN,0,10.31,19.26,0.0,NaN,1.0,0.00,0.00,0.00,0.07,0.0,0.08,0.00,0.06,0.0,0.01,0.00,0.08,0.00,0.08,0.01,0.07,0.0,0.00,0.00,0.10,0.99,0.93,Extracellular,Signal peptide,0.785,0.995,0.679,0.992,1.000,1.0,NaN,NaN,NaN
2,sscle_09g074030,3.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,0.96,0.65,0.80,NaN,0.51,0.67,0.59,24316.53,222,2.5,7.633,5,71,125,64,22,119,50,27,23,1.235,LQKR:[LIJVAP]X[KRTPEI]R&LXXR:66-69|KR:[KR]R:68...,KRIA:81-84|KNLK:114-117|KGYH:152-155,0,...,NaN,0,14.50,26.24,0.0,NaN,1.0,0.00,0.00,0.00,0.07,0.0,0.08,0.00,0.05,0.0,0.01,0.00,0.08,0.00,0.10,0.01,0.07,0.0,0.00,0.00,0.09,0.99,0.93,Extracellular,Signal peptide,0.831,0.996,0.736,0.994,1.000,1.0,NaN,NaN,NaN
3,sscle_05g045060,3.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.97,NaN,0.87,NaN,0.20,0.37,0.89,9667.92,93,0.0,6.394,10,45,61,27,10,59,15,8,7,0.786,NaN,NaN,0,...,NaN,0,2.12,2.14,0.0,NaN,1.0,0.00,0.00,0.00,0.05,0.0,0.07,0.00,0.05,0.0,0.00,0.00,0.04,0.00,0.07,0.00,0.09,0.0,0.00,0.00,0.04,1.00,0.96,Extracellular,Signal peptide,0.871,0.999,0.874,1.000,1.000,1.0,NaN,NaN,SsINE5|BcSSP2
4,sscle_01g003850,3.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.80,0.92,NaN,0.79,NaN,0.28,0.24,0.62,17982.50,163,-4.5,4.442,8,53,91,51,16,91,26,11,15,1.276,LEPR:[LIJVAP]X[KRTPEI]R&LXXR:20-23,NaN,0,...,NaN,0,0.36,2.06,0.0,NaN,1.0,0.00,0.00,0.00,0.09,0.0,0.12,0.00,0.16,0.0,0.02,0.02,0.16,0.00,0.20,0.02,0.19,0.0,0.01,0.00,0.06,0.95,0.87,Extracellular,Signal peptide,0.812,1.000,0.781,0.999,1.000,1.0,NaN,NaN,SsSSVP1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11125,sscle_13g093100,-3.57,LmA

In [70]:
preds.to_csv("effector_preds2.tsv", sep="\t", index=False)

In [71]:
preds

,name,effector_score,phibase_genes,phibase_phenotypes,dbcan_accession,dbcan_description,swissprot_protein_name,swissprot_gene_name,swissprot_accession,pdb_protein_name,pdb_accession,go_accession,go_name,ipr_accession,ipr_name,effectorp1,effectorp2,effectorp3_cytoplasmic,effectorp3_apoplastic,effectorp3_noneffector,deepredeff_fungi,deepredeff_oomycete,apoplastp,molecular_weight,residue_number,charge,isoelectric_point,aa_c_number,aa_tiny_number,aa_small_number,aa_aliphatic_number,aa_aromatic_number,aa_nonpolar_number,aa_charged_number,aa_basic_number,aa_acidic_number,fykin_gap,kex2_cutsites,rxlr_like_motifs,localizer_nucleus,...,phobius_tm_domains,tmhmm_tmcount,tmhmm_first_60,tmhmm_exp_aa,tmhmm_first_tm_sp_coverage,tmhmm_domains,targetp_secreted_prob,targetp_mitochondrial_prob,deeploc_membrane,deeploc_nucleus,deeploc2_nucleus,deeploc_cytoplasm,deeploc2_cytoplasm,deeploc_mitochondrion,deeploc2_mitochondrion,deeploc_plastid,deeploc2_plastid,deeploc_endoplasmic_reticulum,deeploc2_endoplasmic_reticulum,deeploc_golgi,deeploc2_golgi,deeploc_lysosome,deeploc2_lysosome,deeploc_peroxisome,deeploc2_peroxisome,deeploc_cell_membrane,deeploc2_cell_membrane,deeploc_extracellular,deeploc2_extracellular,deeploc2_preds,deeploc2_signals,signalp3_nn_d,signalp3_hmm_s,signalp4_d,signalp5_prob,signalp6_prob,deepsig_signal_prob,deepsig_transmembrane_prob,deepsig_other_prob,effectordb_name
0,sscle_03g031910,3.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,0.97,0.70,0.81,NaN,0.61,0.67,0.42,24242.45,222,2.5,7.633,5,72,124,66,22,121,50,27,23,1.185,LQKR:[LIJVAP]X[KRTPEI]R&LXXR:66-69|KR:[KR]R:68...,KRIA:81-84|KNLK:114-117|KGYH:152-155,0,...,NaN,0,8.77,16.65,0.0,NaN,1.0,0.00,0.00,0.00,0.07,0.0,0.07,0.00,0.06,0.0,0.01,0.00,0.08,0.00,0.08,0.01,0.07,0.0,0.00,0.00,0.10,0.98,0.93,Extracellular,Signal peptide,0.776,0.996,0.677,0.990,1.000,1.0,NaN,NaN,NaN
1,sscle_16g107730,3.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.96,0.71,0.80,NaN,0.61,0.67,0.42,24212.42,222,2.5,7.633,5,72,124,67,22,122,50,27,23,1.164,LQKR:[LIJVAP]X[KRTPEI]R&LXXR:66-69|KR:[KR]R:68...,KRIA:81-84|KNLK:114-117|KGYH:152-155,0,...,NaN,0,10.31,19.26,0.0,NaN,1.0,0.00,0.00,0.00,0.07,0.0,0.08,0.00,0.06,0.0,0.01,0.00,0.08,0.00,0.08,0.01,0.07,0.0,0.00,0.00,0.10,0.99,0.93,Extracellular,Signal peptide,0.785,0.995,0.679,0.992,1.000,1.0,NaN,NaN,NaN
2,sscle_09g074030,3.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,0.96,0.65,0.80,NaN,0.51,0.67,0.59,24316.53,222,2.5,7.633,5,71,125,64,22,119,50,27,23,1.235,LQKR:[LIJVAP]X[KRTPEI]R&LXXR:66-69|KR:[KR]R:68...,KRIA:81-84|KNLK:114-117|KGYH:152-155,0,...,NaN,0,14.50,26.24,0.0,NaN,1.0,0.00,0.00,0.00,0.07,0.0,0.08,0.00,0.05,0.0,0.01,0.00,0.08,0.00,0.10,0.01,0.07,0.0,0.00,0.00,0.09,0.99,0.93,Extracellular,Signal peptide,0.831,0.996,0.736,0.994,1.000,1.0,NaN,NaN,NaN
3,sscle_05g045060,3.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.97,NaN,0.87,NaN,0.20,0.37,0.89,9667.92,93,0.0,6.394,10,45,61,27,10,59,15,8,7,0.786,NaN,NaN,0,...,NaN,0,2.12,2.14,0.0,NaN,1.0,0.00,0.00,0.00,0.05,0.0,0.07,0.00,0.05,0.0,0.00,0.00,0.04,0.00,0.07,0.00,0.09,0.0,0.00,0.00,0.04,1.00,0.96,Extracellular,Signal peptide,0.871,0.999,0.874,1.000,1.000,1.0,NaN,NaN,SsINE5|BcSSP2
4,sscle_01g003850,3.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.80,0.92,NaN,0.79,NaN,0.28,0.24,0.62,17982.50,163,-4.5,4.442,8,53,91,51,16,91,26,11,15,1.276,LEPR:[LIJVAP]X[KRTPEI]R&LXXR:20-23,NaN,0,...,NaN,0,0.36,2.06,0.0,NaN,1.0,0.00,0.00,0.00,0.09,0.0,0.12,0.00,0.16,0.0,0.02,0.02,0.16,0.00,0.20,0.02,0.19,0.0,0.01,0.00,0.06,0.95,0.87,Extracellular,Signal peptide,0.812,1.000,0.781,0.999,1.000,1.0,NaN,NaN,SsSSVP1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11125,sscle_13g093100,-3.57,LmA